# Installing environment

In [1]:
!pip install beautifulsoup4==4.12.3
!pip install requests==2.31.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.31.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# subprocess to download files from GitHub
import subprocess

url = "https://raw.githubusercontent.com/Denis2054/RAG-Driven-Generative-AI/main/commons/grequests.py"
output_file = "grequests.py"

# curl - using private token
curl_command = [
    "curl",
    "-o", output_file,
    url
]

try:
  subprocess.run(curl_command, check=True)
  print("Download successful.")
except subprocess.CalledProcessError:
  print("Failed to download the file.")

Download successful.


In [8]:
import subprocess
import os

# add private token after file name IF necessary
def download(directory, filename):
  base_url = 'https://raw.githubusercontent.com/Denis2054/RAG-DrivenGenerative-AI/main/'

  file_url = f"{base_url}{directory}/{filename}"

  # downloading the file
  try:
    curl_command = f'curl -H -o {filename} {file_url}'

    subprocess.run(curl_command, check=True, shell=True)
    print(f"Downloaded '{filename}' successfully.")
  except subprocess.CalledProcessError:
    print(f"Failed to download '{filename}'. Check URL, connection or if the token is correct/has appropiate permissions.")

In [10]:
!pip install deeplake
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 48.7 MB/s eta 0:00:00


In [11]:
with open('/etc/resolv.conf', 'w') as file:
  file.write("nameserver 8.8.8.8")

In [12]:
# OpenAI auth
import os
import openai

f = open("drive/MyDrive/api_key.txt", "r")
API_KEY = f.readline().strip()
f.close()

os.environ['OPENAI_API_KEY'] = API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [14]:
# Active loop auth for deep lake
f = open("drive/MyDrive/activeloop_key.txt", "r")
API_token=f.readline().strip()
f.close()
ACTIVELOOP_TOKEN=API_token
os.environ['ACTIVELOOP_TOKEN'] =ACTIVELOOP_TOKEN


# Data collection and preparation

### Collecting the data

In [15]:
import requests
from bs4 import BeautifulSoup
import re

In [16]:
# URLS of wiki articles

urls = [
  "https://en.wikipedia.org/wiki/Space_exploration",
  "https://en.wikipedia.org/wiki/Apollo_program",
  "https://en.wikipedia.org/wiki/Hubble_Space_Telescope",
  "https://en.wikipedia.org/wiki/Mars_over",
  "https://en.wikipedia.org/wiki/International_Space_Station",
  "https://en.wikipedia.org/wiki/SpaceX",
  "https://en.wikipedia.org/wiki/Juno_(spacecraft)",
  "https://en.wikipedia.org/wiki/Voyager_program",
  "https://en.wikipedia.org/wiki/Galileo_(spacecraft)",
  "https://en.wikipedia.org/wiki/Kepler_Space_Telescope"
 ]

### Preparing the data

In [30]:
def clean_text(content: str) -> str:
  # remove references
  content = re.sub(r'\[\d+\]', '', content)
  # remove citations
  content = re.sub(r'\([^)]*\)', '', content)
  # remove unwanted characters
  content = re.sub(r'[^a-zA-Z0-9\s]', '', content)
  # remove extra whitespaces
  content = re.sub(r'\s+', ' ', content)
  return content

In [31]:
def fetch_and_clean(url: str) -> str:
  # fetch the content
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  # finding the main content of the article
  content = soup.find('div', {'class': 'mw-parser-output'})

  # remove the bibliography section
  for section_title in ['References', 'Bibliography', 'External links', 'See also']:
    section = content.find('span', id=section_title)
    if section:
      # remove all content from this section
      for sib in section.parent.find_next_siblings():
        sib.decompose()
      section.parent.decompose()

  # Extract text and clean it
  text = content.get_text(separator=' ', strip=True)
  text = clean_text(text)
  return text

In [32]:
# file to storage the cleaned text
with open('llm.txt', 'w', encoding='utf-8') as file:
  for url in urls:
    clean_article_text = fetch_and_clean(url)
    file.write(clean_article_text + '\n')

print("Content written to 'llm.txt'")

Content written to 'llm.txt'


In [33]:
# open and print first 30 lines of the file
with open('llm.txt', 'r', encoding='utf-8') as file:
  lines = file.readlines()

  for line in lines[:30]:
    print(line.strip())

Exploration of space planets and moons Space Exploration redirects here For the company see SpaceX For broader coverage of this topic see Exploration Buzz Aldrin taking a core sample of the Moon during the Apollo 11 mission Selfportrait of Curiosity rover on Mars s surface Part of a series on Spaceflight History History of spaceflight Space Race Timeline of spaceflight Space probes Lunar missions Mars missions Applications Communications Earth observation Exploration Espionage Military Navigation Colonization Habitation Exploration Telescopes Tourism Spacecraft Robotic spacecraft Satellite Space probe Cargo spacecraft Crewed spacecraft Apollo Lunar Module Space capsules Space Shuttle Space stations Spaceplanes Vostok Space launch Spaceport Launch pad Expendable and reusable launch vehicles Escape velocity Nonrocket spacelaunch Spaceflight types Suborbital Orbital Interplanetary Interstellar Intergalactic List of space organizations Space agencies Space forces Companies Spaceflight port